The Fast Gradient Sign Method (FGSM) - это простой, но эффективный метод для генерации состязательного изображения. 
Этот метод был представлен в статье [Explaining and Harnessing Adversarial Examples](https://arxiv.org/abs/1412.6572) в 2014 году. 

Алгоритм FGSM
1. Берём исходное изображение 
2. Получаем предсказание, используя обученную CNN 
3. Вычисляем значение функции потерь 
4. Вычисляем градиент по отношению к исходному изображению 
5. Вычисляем знак градиента 
6. С использование знакового градиента генерирует состязательную картинку с шумами 

<center>Формула для FGSM</center>

$$ \text{adv\_x} = x + \epsilon * sign(\nabla_xJ(\theta, x, y)) $$

где 
* $\text{adv\_x}$ - наша выходная картинка с шумами 
* $x$ - исходное входное изображение 
* $y$ - истинные классы для входного изображения 
* $\epsilon$ - малый коэффициент, на который мы умножаем знаковый градиент. Это значение гарантирует, что полученные шумы будут достаточно малы, чтобы человеческий глаз не заметил подвоха 
* $\theta$ - нейронная сеть 
* $J$ - функция потерь

In [ ]:
%load_ext autoreload
%autoreload 2

import os 

import matplotlib.pyplot as plt 
import numpy as np

import torch 
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision.transforms import transforms

random_seed = 42
torch.backends.cudnn.enabled = False
torch.manual_seed(random_seed)

CURRENT_DIR = os.path.dirname(os.path.abspath("__file__"))
CV_DIR = os.path.abspath(os.path.join(CURRENT_DIR, os.pardir))

In [ ]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

DEVICE

In [ ]:
# загружаем тестовую выборку
test_data = torchvision.datasets.MNIST(
    "mnist_content", train=False, transform=transforms.Compose([transforms.ToTensor()]), download=True
)

test_dataloader=torch.utils.data.DataLoader(
    dataset=test_data, 
    batch_size=1,
    shuffle=False
)

Загружаем нашу обученную модель (модель была обучена в ноутбуке - `recognition/XX_MNIST.ipynb`)

In [ ]:
class MNIST(nn.Module):
    def __init__(self, input_size, hidden_size1, hidden_size2, hidden_size3, hidden_size, output):
        super().__init__()
        self.f_connected1=nn.Linear(input_size, hidden_size1)
        self.f_connected2=nn.Linear(hidden_size1, hidden_size2)
        self.f_connected3=nn.Linear(hidden_size2, hidden_size3)
        self.f_connected4=nn.Linear(hidden_size3, hidden_size)
        self.out_connected=nn.Linear(hidden_size, output)

    def forward(self,x):
            out=F.relu(self.f_connected1(x)) 
            out=F.relu(self.f_connected2(out))
            out=F.relu(self.f_connected3(out))
            out=F.relu(self.f_connected4(out))
            out=self.out_connected(out)
            return out

CONFIG = { 
    "input_size": 784,      # 28x28 
    "hidden_size_1": 200,   # размер 1-го скрытого слоя
    "hidden_size_2": 150,   # размер 2-го скрытого слоя
    "hidden_size_3": 100,   # размер 3-го скрытого слоя
    "hidden_size_4": 80,    # размер 4-го скрытого слоя 
    "output": 10,           # кол-во выходов сети (т.к. цифры от 0 до 9)
    "bach_size": 132,       # обычно используется степень 2-ки
    "lr_rate": 0.01
}

model = MNIST(
    input_size=CONFIG["input_size"], 
    hidden_size1=CONFIG["hidden_size_1"],
    hidden_size2=CONFIG["hidden_size_2"], 
    hidden_size3=CONFIG["hidden_size_3"], 
    hidden_size=CONFIG["hidden_size_4"], 
    output=CONFIG["output"]
)
model = model.to(DEVICE)

MODEL_FPATH = os.path.join(CV_DIR, "recognition", "mnist_checkpoints", "best.pth")
model.load_state_dict(torch.load(MODEL_FPATH)["model_state"])

model.eval()

In [ ]:
def fgsm_attack(image, epsilon, data_grad):
    # Collect the element-wise sign of the data gradient
    sign_data_grad = data_grad.sign()
    # Create the perturbed image by adjusting each pixel of the input image
    perturbed_image = image + epsilon*sign_data_grad
    # Adding clipping to maintain [0,1] range
    perturbed_image = torch.clamp(perturbed_image, 0, 1)
    # Return the perturbed image
    return perturbed_image

In [ ]:
def test_attack( model, device, test_loader, epsilon ):
    # Accuracy counter
    correct = 0
    adv_examples = []

    # Loop over all examples in test set
    for data, target in test_loader:
        # Send the data and label to the device
        data, target = data.to(device), target.to(device)

        # Set requires_grad attribute of tensor. Important for Attack
        data.requires_grad = True

        # Forward pass the data through the model
        output = model(data.reshape(-1, 28 * 28))
        init_pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability

        # If the initial prediction is wrong, dont bother attacking, just move on
        if init_pred.item() != target.item():
            continue

        # Calculate the loss
        loss = F.nll_loss(output, target)

        # Zero all existing gradients
        model.zero_grad()

        # Calculate gradients of model in backward pass
        loss.backward()

        # Collect datagrad
        data_grad = data.grad.data

        # Call FGSM Attack
        perturbed_data = fgsm_attack(data.reshape(-1, 28 * 28), epsilon, data_grad.reshape(-1, 28 * 28))

        # Re-classify the perturbed image
        output = model(perturbed_data)

        # Check for success
        final_pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability
        if final_pred.item() == target.item():
            correct += 1
            # Special case for saving 0 epsilon examples
            if (epsilon == 0) and (len(adv_examples) < 5):
                adv_ex = perturbed_data.squeeze().detach().cpu().numpy()
                adv_examples.append( (init_pred.item(), final_pred.item(), adv_ex) )
        else:
            # Save some adv examples for visualization later
            if len(adv_examples) < 5:
                adv_ex = perturbed_data.squeeze().detach().cpu().numpy()
                adv_examples.append( (init_pred.item(), final_pred.item(), adv_ex) )

    # Calculate final accuracy for this epsilon
    final_acc = correct/float(len(test_loader))
    print(f"Epsilon: {epsilon}\tTest Accuracy = {correct} / {len(test_loader)} = {final_acc}")

    # Return the accuracy and an adversarial example
    return final_acc, adv_examples

In [ ]:
epsilons = [0, .05, .1, .15, .2, .25, .3]

accuracies = []
examples = []

for eps in epsilons:
    acc, ex = test_attack(model, DEVICE, test_dataloader, eps)
    accuracies.append(acc)
    examples.append(ex)

In [ ]:
plt.figure(figsize=(12,6))
plt.plot(epsilons, accuracies, "*-")
plt.yticks(np.arange(0, 1.1, step=0.1))
plt.xticks(np.arange(0, .35, step=0.05))
plt.title("Accuracy vs Epsilon")
plt.xlabel("Epsilon")
plt.ylabel("Accuracy")
plt.grid(True)
plt.show()

In [ ]:
# Plot several examples of adversarial samples at each epsilon
cnt = 0
plt.figure(figsize=(8,10))
for i in range(len(epsilons)):
    for j in range(len(examples[i])):
        cnt += 1

        plt.subplot(len(epsilons),len(examples[0]),cnt)
        plt.xticks([], [])
        plt.yticks([], [])
        if j == 0:
            plt.ylabel(f"Eps: {epsilons[i]}", fontsize=14)
        orig, adv, ex = examples[i][j]
        plt.title(f"{orig} -> {adv}")
        plt.imshow(ex.reshape(28, 28), cmap="gray")
plt.tight_layout()
plt.show()

## Полезные ссылки 

* [FGSM Tutorial](https://pytorch.org/tutorials/beginner/fgsm_tutorial.html)